In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import warnings 
import geopandas as gpd
import re
from mpl_toolkits.axes_grid1 import make_axes_locatable
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight') 

# Set some parameters
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.size'] = 14
np.set_printoptions(4)

In [12]:
pip install CensusData

Note: you may need to restart the kernel to use updated packages.


In [13]:
import censusdata
#pd.set_option('display.expand_frame_repr', False)
#pd.set_option('display.precision', 2)

In [14]:
housing = censusdata.search('acs5', 2015, 'label', 'housing')
income = censusdata.search('acs5', 2015, 'label',  'income')
poverty = censusdata.search('acs5', 2015, 'label',  'poverty')
earnings = censusdata.search('acs5', 2015, 'label',  'earnings')
group_quarters = censusdata.search('acs5', 2015, 'label',  'group quarters')

In [15]:
ca_county_lst = []
for i in range(1, 113, 2):
    if i in range(1,10):
        ca_county_lst.append('00' + str(i))
    elif i in range(11,100):
        ca_county_lst.append('0' + str(i))
    else:
        ca_county_lst.append(str(i))
ca_county_lst

['001',
 '003',
 '005',
 '007',
 '009',
 '011',
 '013',
 '015',
 '017',
 '019',
 '021',
 '023',
 '025',
 '027',
 '029',
 '031',
 '033',
 '035',
 '037',
 '039',
 '041',
 '043',
 '045',
 '047',
 '049',
 '051',
 '053',
 '055',
 '057',
 '059',
 '061',
 '063',
 '065',
 '067',
 '069',
 '071',
 '073',
 '075',
 '077',
 '079',
 '081',
 '083',
 '085',
 '087',
 '089',
 '091',
 '093',
 '095',
 '097',
 '099',
 '101',
 '103',
 '105',
 '107',
 '109',
 '111']

In [16]:
censusdata.printtable(censusdata.censustable('acs5', 2015, 'B25024'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B25024_001E  | B25024.  Units in Structure    | Total:                                                   | int  
B25024_002E  | B25024.  Units in Structure    | 1, detached                                              | int  
B25024_003E  | B25024.  Units in Structure    | 1, attached                                              | int  
B25024_004E  | B25024.  Units in Structure    | 2                                                        | int  
B25024_005E  | B25024.  Units in Structure    | 3 or 4                                                   | int  
B25024_006E  | B25024.  Units in Structure    | 5 to 9                                                   | int  
B25024_007E  | B25024.  Units in Structure    | 10 to 19                                     

In [17]:
units_in_structure_df_names = {}
for i in range(len(ca_county_lst)):
    df_name = 'units_in_structure_df_county_' + str(ca_county_lst[i])
    
    df_data = censusdata.download('acs5', 2015,
                             censusdata.censusgeo([('state', '06'), ('county', ca_county_lst[i]), ('block group', '*')]),
                             ['B25024_002E', 'B25024_003E', 'B25024_004E','B25024_005E', 'B25024_006E', 'B25024_007E'
                              ,'B25024_008E', 'B25024_009E'])
    
    df_data = df_data.rename(columns={'B25024_002E': '1, detached', 'B25024_003E':'1, attached ', 'B25024_004E':'2',
                                        'B25024_005E':'3 or 4', 'B25024_006E':'5 to 9', 'B25024_007E':'10 to 19', 
                                        'B25024_008E':'20 to 49','B25024_009E':'50 or more'})
    df_data = df_data.div(df_data.sum(axis=1), axis=0)
    
    for i in range(len(df_data.index.values)):
        df_data.index.values[i] = str(df_data.index.values[i])[:31]
    
    units_in_structure_df_names[df_name] = df_data
units_in_structure_df_names['units_in_structure_df_county_001']

,"1, detached","1, attached",2,3 or 4,5 to 9,10 to 19,20 to 49,50 or more
"Block Group 4, Census Tract 409",0.619048,0.000000,0.000000,0.258503,0.122449,0.000000,0.000000,0.000000
"Block Group 1, Census Tract 409",1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
"Block Group 2, Census Tract 409",0.479472,0.000000,0.016129,0.209677,0.108504,0.057185,0.085044,0.043988
"Block Group 3, Census Tract 409",1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
"Block Group 1, Census Tract 409",0.819608,0.074510,0.000000,0.105882,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
"Block Group 2, Census Tract 450",0.896552,0.061782,0.000000,0.000000,0.000000,0.000000,0.020115,0.021552
"Block Group 1, Census Tract 450",0.615010,0.122807,0.000000,0.063353,0.112086,0.025341,0.000000,0.061404
"Block Group 2, Census Tract 450",0.693662,0.125000,0.000000,0.004401,0.014965,0.022887,0.056338,0.082746
"Block Group 1, Census Tract 450",0.614907,0.230849,0.052795,0.015528,0.008282,0.000000,0.037267,0.040373


In [18]:
censusdata.printtable(censusdata.censustable('acs5', 2015, 'B25056'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B25056_001E  | B25056. Contract Rent          | Total:                                                   | int  
B25056_002E  | B25056. Contract Rent          | With cash rent:                                          | int  
B25056_003E  | B25056. Contract Rent          | !! With cash rent: Less than $100                        | int  
B25056_004E  | B25056. Contract Rent          | !! With cash rent: $100 to $149                          | int  
B25056_005E  | B25056. Contract Rent          | !! With cash rent: $150 to $199                          | int  
B25056_006E  | B25056. Contract Rent          | !! With cash rent: $200 to $249                          | int  
B25056_007E  | B25056. Contract Rent          | !! With cash rent: $250 to $299              

In [19]:
contract_rent_df_names = {}
for i in range(len(ca_county_lst)):
    df_name = 'contract_rent_df_county_' + str(ca_county_lst[i])
    
    df_data = censusdata.download('acs5', 2015,
                             censusdata.censusgeo([('state', '06'), ('county', ca_county_lst[i]), ('block group', '*')]),
                             ['B25056_003E', 'B25056_004E', 'B25056_005E', 'B25056_006E',
                              'B25056_007E', 'B25056_008E','B25056_009E', 'B25056_010E','B25056_011E', 'B25056_012E',
                              'B25056_013E', 'B25056_014E','B25056_015E', 'B25056_016E','B25056_017E', 'B25056_018E',
                              'B25056_019E', 'B25056_020E','B25056_021E', 'B25056_022E','B25056_023E', 'B25056_024E', 
                              'B25056_025E', 'B25056_026E'])
    
    df_data = df_data.rename(columns={'B25056_003E':'With cash rent: Less than \$100', 'B25056_004E':'With cash rent: \$100 to \$149',
                               'B25056_005E':'With cash rent: \$150 to \$199', 'B25056_006E':'With cash rent: \$200 to \$249',
                              'B25056_007E':'With cash rent: \$250 to \$299', 'B25056_008E':'With cash rent: \$300 to \$349',
                              'B25056_009E':'With cash rent: \$350 to \$399', 'B25056_010E':'With cash rent: \$400 to \$449',
                               'B25056_011E':'With cash rent: \$450 to \$499', 'B25056_012E':'With cash rent: \$500 to \$549',
                              'B25056_013E':'With cash rent: \$550 to \$599', 'B25056_014E':'With cash rent: \$600 to \$649',
                               'B25056_015E':'With cash rent: \$650 to \$699', 'B25056_016E':'With cash rent: \$700 to \$749',
                              'B25056_017E':'With cash rent: \$750 to \$799', 'B25056_018E':'With cash rent: \$800 to \$899',
                              'B25056_019E':'With cash rent: \$900 to \$999', 'B25056_020E':'With cash rent: \$1,000 to \$1,249',
                              'B25056_021E':'With cash rent: \$1,250 to \$1,499', 'B25056_022E':'With cash rent: \$1,500 to \$1,999 ',
                               'B25056_023E':'With cash rent: \$2,000 to \$2,499', 'B25056_024E':'With cash rent: \$2,500 to \$2,999', 
                              'B25056_025E':'With cash rent: \$3,000 to \$3,499', 'B25056_026E':'With cash rent: \$3,500 or more'})
    df_data = df_data.div(df_data.sum(axis=1), axis=0)
        
    for i in range(len(df_data.index.values)):
        df_data.index.values[i] = str(df_data.index.values[i])[:31]
    
    contract_rent_df_names[df_name] = df_data
contract_rent_df_names['contract_rent_df_county_001']

,With cash rent: Less than \$100,With cash rent: \$100 to \$149,With cash rent: \$150 to \$199,With cash rent: \$200 to \$249,With cash rent: \$250 to \$299,With cash rent: \$300 to \$349,With cash rent: \$350 to \$399,With cash rent: \$400 to \$449,With cash rent: \$450 to \$499,With cash rent: \$500 to \$549,...,With cash rent: \$750 to \$799,With cash rent: \$800 to \$899,With cash rent: \$900 to \$999,"With cash rent: \$1,000 to \$1,249","With cash rent: \$1,250 to \$1,499","With cash rent: \$1,500 to \$1,999","With cash rent: \$2,000 to \$2,499","With cash rent: \$2,500 to \$2,999","With cash rent: \$3,000 to \$3,499","With cash rent: \$3,500 or more"
"Block Group 4, Census Tract 409",0.000000,0.000000,0.0,0.067568,0.000000,0.000000,0.0,0.047297,0.000000,0.000000,...,0.000000,0.000000,0.020270,0.385135,0.256757,0.175676,0.000000,0.000000,0.000000,0.000000
"Block Group 1, Census Tract 409",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.614583,0.000000,0.000000,0.093750,0.291667,0.000000,0.000000
"Block Group 2, Census Tract 409",0.032663,0.032663,0.0,0.075377,0.032663,0.022613,0.0,0.000000,0.027638,0.090452,...,0.115578,0.133166,0.020101,0.173367,0.000000,0.165829,0.027638,0.000000,0.000000,0.000000
"Block Group 3, Census Tract 409",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.200000,0.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
"Block Group 1, Census Tract 409",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Block Group 2, Census Tract 450",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.606557,0.000000,0.000000,0.114754,0.278689,0.000000,0.000000
"Block Group 1, Census Tract 450",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.038806,0.000000,0.000000,0.026866,0.104478,0.349254,0.208955,0.152239,0.074627,0.000000
"Block Group 2, Census Tract 450",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.318052,0.000000,0.071633,0.103152,0.200573,0.140401,0.000000,0.000000,0.040115
"Block Group 1, Census Tract 450",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.033654,0.403846,0.067308,0.000000,0.206731,0.139423,0.110577,0.038462,0.000000


In [20]:
censusdata.printtable(censusdata.censustable('acs5', 2015, 'B25034'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B25034_001E  | B25034. Year Structure Built   | Total:                                                   | int  
B25034_002E  | B25034. Year Structure Built   | Built 2014 or later                                      | int  
B25034_003E  | B25034. Year Structure Built   | Built 2010 to 2013                                       | int  
B25034_004E  | B25034. Year Structure Built   | Built 2000 to 2009                                       | int  
B25034_005E  | B25034. Year Structure Built   | Built 1990 to 1999                                       | int  
B25034_006E  | B25034. Year Structure Built   | Built 1980 to 1989                                       | int  
B25034_007E  | B25034. Year Structure Built   | Built 1970 to 1979                           

In [21]:
year_structure_built_df = censusdata.download('acs5', 2015,
                             censusdata.censusgeo([('state', '06'), ('county', '001'), ('block group', '*')]),
                             ['B25034_001E', 'B25034_002E', 'B25034_003E', 'B25034_004E', 'B25034_005E', 'B25034_006E',
                              'B25034_007E', 'B25034_008E','B25034_009E', 'B25034_010E','B25034_011E'])

year_structure_built_df.head()

,B25034_001E,B25034_002E,B25034_003E,B25034_004E,B25034_005E,B25034_006E,B25034_007E,B25034_008E,B25034_009E,B25034_010E,B25034_011E
"Block Group 4, Census Tract 4097, Alameda County, California: Summary level: 150, state:06> county:001> tract:409700> block group:4",294,0,0,30,0,24,45,59,32,50,54
"Block Group 1, Census Tract 4098, Alameda County, California: Summary level: 150, state:06> county:001> tract:409800> block group:1",363,0,0,0,22,0,0,70,101,39,131
"Block Group 2, Census Tract 4098, Alameda County, California: Summary level: 150, state:06> county:001> tract:409800> block group:2",682,0,0,30,62,13,133,130,141,129,44
"Block Group 3, Census Tract 4098, Alameda County, California: Summary level: 150, state:06> county:001> tract:409800> block group:3",279,0,0,29,0,8,18,70,88,45,21
"Block Group 1, Census Tract 4099, Alameda County, California: Summary level: 150, state:06> county:001> tract:409900> block group:1",255,0,0,0,0,20,10,58,119,28,20


In [22]:
year_structure_built_df_names = {}
for i in range(len(ca_county_lst)):
    df_name = 'year_structure_built_df_county_' + str(ca_county_lst[i])
    
    df_data = censusdata.download('acs5', 2015,
                             censusdata.censusgeo([('state', '06'), ('county', ca_county_lst[i]), ('block group', '*')]),
                             ['B25034_002E', 'B25034_003E', 'B25034_004E', 'B25034_005E', 'B25034_006E',
                              'B25034_007E', 'B25034_008E','B25034_009E', 'B25034_010E','B25034_011E'])
    
    df_data = df_data.rename(columns={'B25034_002E':'Built 2014 or later', 'B25034_003E':'Built 2010 to 2013', 
                                      'B25034_004E':'Built 2000 to 2009', 'B25034_005E':'Built 1990 to 1999', 
                                      'B25034_006E':'Built 1980 to 1989', 'B25034_007E':'Built 1970 to 1979', 
                                      'B25034_008E':'Built 1960 to 1969','B25034_009E':'1950 to 1959', 
                                      'B25034_010E':'1940 to 1949','B25034_011E':'Built 1939 or earlier'})
    df_data = df_data.div(df_data.sum(axis=1), axis=0)
        
    for i in range(len(df_data.index.values)):
        df_data.index.values[i] = str(df_data.index.values[i])[:31]
    
    year_structure_built_df_names[df_name] = df_data
year_structure_built_df_names['year_structure_built_df_county_001']

,Built 2014 or later,Built 2010 to 2013,Built 2000 to 2009,Built 1990 to 1999,Built 1980 to 1989,Built 1970 to 1979,Built 1960 to 1969,1950 to 1959,1940 to 1949,Built 1939 or earlier
"Block Group 4, Census Tract 409",0.0,0.000000,0.102041,0.000000,0.081633,0.153061,0.200680,0.108844,0.170068,0.183673
"Block Group 1, Census Tract 409",0.0,0.000000,0.000000,0.060606,0.000000,0.000000,0.192837,0.278237,0.107438,0.360882
"Block Group 2, Census Tract 409",0.0,0.000000,0.043988,0.090909,0.019062,0.195015,0.190616,0.206745,0.189150,0.064516
"Block Group 3, Census Tract 409",0.0,0.000000,0.103943,0.000000,0.028674,0.064516,0.250896,0.315412,0.161290,0.075269
"Block Group 1, Census Tract 409",0.0,0.000000,0.000000,0.000000,0.078431,0.039216,0.227451,0.466667,0.109804,0.078431
...,...,...,...,...,...,...,...,...,...,...
"Block Group 2, Census Tract 450",0.0,0.041076,0.060907,0.175637,0.573654,0.092068,0.015581,0.028329,0.000000,0.012748
"Block Group 1, Census Tract 450",0.0,0.000000,0.105263,0.690058,0.147173,0.024366,0.018519,0.000000,0.000000,0.014620
"Block Group 2, Census Tract 450",0.0,0.000000,0.251761,0.214789,0.419014,0.022887,0.036972,0.039613,0.000000,0.014965
"Block Group 1, Census Tract 450",0.0,0.000000,0.024465,0.261978,0.389399,0.196738,0.072375,0.046891,0.000000,0.008155


In [23]:
censusdata.printtable(censusdata.censustable('acs5', 2015, 'B25038'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B25038_001E  | B25038. Tenure by Year Househo | Total:                                                   | int  
B25038_002E  | B25038. Tenure by Year Househo | Owner occupied:                                          | int  
B25038_003E  | B25038. Tenure by Year Househo | !! Owner occupied: Moved in 2015 or later                | int  
B25038_004E  | B25038. Tenure by Year Househo | !! Owner occupied: Moved in 2010 to 2014                 | int  
B25038_005E  | B25038. Tenure by Year Househo | !! Owner occupied: Moved in 2000 to 2009                 | int  
B25038_006E  | B25038. Tenure by Year Househo | !! Owner occupied: Moved in 1990 to 1999                 | int  
B25038_007E  | B25038. Tenure by Year Househo | !! Owner occupied: Moved in 1980 to 1989     

In [24]:
year_moved_in_df_names = {}
for i in range(len(ca_county_lst)):
    df_name = 'year_moved_in_df_county_' + str(ca_county_lst[i])
    
    df_data = censusdata.download('acs5', 2015,
                             censusdata.censusgeo([('state', '06'), ('county', ca_county_lst[i]), ('block group', '*')]),
                             ['B25038_003E', 'B25038_004E', 'B25038_005E', 'B25038_006E', 'B25038_007E', 'B25038_008E',
                              'B25038_010E', 'B25038_011E', 'B25038_012E', 'B25038_013E', 'B25038_014E', 'B25038_015E'])
    
    df_data = df_data.rename(columns={'B25038_003E':'Owner occupied: Moved in 2015 or later', 'B25038_004E':'Owner occupied: Moved in 2010 to 2014',
                                      'B25038_005E':'Owner occupied: Moved in 2000 to 2009', 'B25038_006E':'Owner occupied: Moved in 1990 to 1999', 
                                      'B25038_007E':'Owner occupied: Moved in 1980 to 1989', 'B25038_008E':'Owner occupied: Moved in 1979 or earlier',
                              'B25038_010E':'Renter occupied: Moved in 2015 or later', 'B25038_011E':'Renter occupied: Moved in 2010 to 2014',
                                      'B25038_012E':'Renter occupied: Moved in 2000 to 2009', 'B25038_013E':'Renter occupied: Moved in 1990 to 1999',
                                      'B25038_014E':'Renter occupied: Moved in 1980 to 1989', 'B25038_015E':'Renter occupied: Moved in 1979 or earlier'})
    df_data = df_data.div(df_data.sum(axis=1), axis=0)
        
    for i in range(len(df_data.index.values)):
        df_data.index.values[i] = str(df_data.index.values[i])[:31]
    
    year_moved_in_df_names[df_name] = df_data
year_moved_in_df_names['year_moved_in_df_county_001']


,Owner occupied: Moved in 2015 or later,Owner occupied: Moved in 2010 to 2014,Owner occupied: Moved in 2000 to 2009,Owner occupied: Moved in 1990 to 1999,Owner occupied: Moved in 1980 to 1989,Owner occupied: Moved in 1979 or earlier,Renter occupied: Moved in 2015 or later,Renter occupied: Moved in 2010 to 2014,Renter occupied: Moved in 2000 to 2009,Renter occupied: Moved in 1990 to 1999,Renter occupied: Moved in 1980 to 1989,Renter occupied: Moved in 1979 or earlier
"Block Group 4, Census Tract 409",0.000000,0.038462,0.203846,0.126923,0.000000,0.061538,0.000000,0.276923,0.292308,0.000000,0.0,0.000000
"Block Group 1, Census Tract 409",0.000000,0.032353,0.285294,0.117647,0.055882,0.214706,0.000000,0.108824,0.000000,0.161765,0.0,0.023529
"Block Group 2, Census Tract 409",0.000000,0.039249,0.023891,0.112628,0.029010,0.116041,0.000000,0.255973,0.383959,0.039249,0.0,0.000000
"Block Group 3, Census Tract 409",0.000000,0.224670,0.096916,0.105727,0.281938,0.057269,0.000000,0.154185,0.039648,0.000000,0.0,0.039648
"Block Group 1, Census Tract 409",0.000000,0.117647,0.164706,0.349020,0.043137,0.243137,0.000000,0.039216,0.000000,0.043137,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
"Block Group 2, Census Tract 450",0.015552,0.107309,0.290824,0.283048,0.194401,0.013997,0.000000,0.069984,0.013997,0.010886,0.0,0.000000
"Block Group 1, Census Tract 450",0.000000,0.098901,0.388611,0.154845,0.000000,0.009990,0.000000,0.271728,0.062937,0.012987,0.0,0.000000
"Block Group 2, Census Tract 450",0.027289,0.127641,0.366197,0.127641,0.044014,0.000000,0.014085,0.114437,0.150528,0.013204,0.0,0.014965
"Block Group 1, Census Tract 450",0.000000,0.079511,0.326198,0.257900,0.084608,0.039755,0.008155,0.114169,0.089704,0.000000,0.0,0.000000


In [25]:
censusdata.printtable(censusdata.censustable('acs5', 2015, 'B25041'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B25041_001E  | B25041.  Bedrooms              | Total:                                                   | int  
B25041_002E  | B25041.  Bedrooms              | No bedroom                                               | int  
B25041_003E  | B25041.  Bedrooms              | 1 bedroom                                                | int  
B25041_004E  | B25041.  Bedrooms              | 2 bedrooms                                               | int  
B25041_005E  | B25041.  Bedrooms              | 3 bedrooms                                               | int  
B25041_006E  | B25041.  Bedrooms              | 4 bedrooms                                               | int  
B25041_007E  | B25041.  Bedrooms              | 5 or more bedrooms                           

In [26]:

num_bedrooms_df_names = {}
for i in range(len(ca_county_lst)):
    df_name = 'num_bedrooms_df_county_' + str(ca_county_lst[i])
    
    df_data = censusdata.download('acs5', 2015,
                             censusdata.censusgeo([('state', '06'), ('county', ca_county_lst[i]), ('block group', '*')]),
                             ['B25041_002E', 'B25041_003E', 'B25041_004E', 'B25041_005E', 'B25041_006E',
                              'B25041_007E'])
    
    df_data = df_data.rename(columns={'B25041_002E':'No bedroom', 'B25041_003E':'1 bedrooms',
                                      'B25041_004E':'2 bedrooms', 'B25041_005E':'3 bedrooms', 'B25041_006E':'4 bedroom',
                              'B25041_007E':'5 or more bedrooms'})
    df_data = df_data.div(df_data.sum(axis=1), axis=0)
        
    for i in range(len(df_data.index.values)):
        df_data.index.values[i] = str(df_data.index.values[i])[:31]
    
    num_bedrooms_df_names[df_name] = df_data
num_bedrooms_df_names['num_bedrooms_df_county_001']


,No bedroom,1 bedrooms,2 bedrooms,3 bedrooms,4 bedroom,5 or more bedrooms
"Block Group 4, Census Tract 409",0.040816,0.010204,0.425170,0.363946,0.159864,0.000000
"Block Group 1, Census Tract 409",0.000000,0.000000,0.239669,0.556474,0.203857,0.000000
"Block Group 2, Census Tract 409",0.016129,0.228739,0.324047,0.310850,0.071848,0.048387
"Block Group 3, Census Tract 409",0.000000,0.032258,0.086022,0.455197,0.351254,0.075269
"Block Group 1, Census Tract 409",0.000000,0.031373,0.149020,0.662745,0.141176,0.015686
...,...,...,...,...,...,...
"Block Group 2, Census Tract 450",0.000000,0.019830,0.082153,0.269122,0.610482,0.018414
"Block Group 1, Census Tract 450",0.000000,0.034113,0.257310,0.385965,0.272904,0.049708
"Block Group 2, Census Tract 450",0.008803,0.144366,0.223592,0.183099,0.338908,0.101232
"Block Group 1, Census Tract 450",0.000000,0.084608,0.259939,0.477064,0.170234,0.008155


In [27]:
censusdata.printtable(censusdata.censustable('acs5', 2015, 'B25017'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B25017_001E  | B25017.  Rooms                 | Total:                                                   | int  
B25017_002E  | B25017.  Rooms                 | 1 room                                                   | int  
B25017_003E  | B25017.  Rooms                 | 2 rooms                                                  | int  
B25017_004E  | B25017.  Rooms                 | 3 rooms                                                  | int  
B25017_005E  | B25017.  Rooms                 | 4 rooms                                                  | int  
B25017_006E  | B25017.  Rooms                 | 5 rooms                                                  | int  
B25017_007E  | B25017.  Rooms                 | 6 rooms                                      

In [28]:
num_rooms_df_names = {}
for i in range(len(ca_county_lst)):
    df_name = 'num_rooms_df_county_' + str(ca_county_lst[i])
    
    df_data = censusdata.download('acs5', 2015,
                             censusdata.censusgeo([('state', '06'), ('county', ca_county_lst[i]), ('block group', '*')]),
                             ['B25017_002E', 'B25017_003E', 'B25017_004E', 'B25017_005E', 'B25017_006E',
                              'B25017_007E', 'B25017_008E', 'B25017_009E', 'B25017_010E'])
    
    df_data = df_data.rename(columns={'B25017_002E':'1 room', 'B25017_003E':'2 rooms', 
                                      'B25017_004E':'3 rooms', 'B25017_005E':'4 rooms', 'B25017_006E':'5 rooms',
                                      'B25017_007E':'6 rooms', 'B25017_008E':'7 rooms', 'B25017_009E':'8 rooms', 
                                      'B25017_010E':'9 or more rooms'})
    df_data = df_data.div(df_data.sum(axis=1), axis=0)
        
    for i in range(len(df_data.index.values)):
        df_data.index.values[i] = str(df_data.index.values[i])[:31]
    
    num_rooms_df_names[df_name] = df_data
num_rooms_df_names['num_rooms_df_county_001']


,1 room,2 rooms,3 rooms,4 rooms,5 rooms,6 rooms,7 rooms,8 rooms,9 or more rooms
"Block Group 4, Census Tract 409",0.040816,0.000000,0.010204,0.391156,0.299320,0.129252,0.129252,0.000000,0.000000
"Block Group 1, Census Tract 409",0.000000,0.000000,0.000000,0.024793,0.146006,0.550964,0.112948,0.132231,0.033058
"Block Group 2, Census Tract 409",0.016129,0.036657,0.178886,0.222874,0.189150,0.153959,0.086510,0.046921,0.068915
"Block Group 3, Census Tract 409",0.000000,0.000000,0.032258,0.025090,0.258065,0.161290,0.293907,0.172043,0.057348
"Block Group 1, Census Tract 409",0.000000,0.000000,0.031373,0.105882,0.086275,0.321569,0.239216,0.031373,0.184314
...,...,...,...,...,...,...,...,...,...
"Block Group 2, Census Tract 450",0.000000,0.000000,0.019830,0.000000,0.209632,0.140227,0.308782,0.215297,0.106232
"Block Group 1, Census Tract 450",0.000000,0.008772,0.096491,0.176413,0.271930,0.120858,0.201754,0.050682,0.073099
"Block Group 2, Census Tract 450",0.000000,0.031690,0.126761,0.136444,0.108275,0.150528,0.222711,0.113556,0.110035
"Block Group 1, Census Tract 450",0.000000,0.016310,0.089704,0.202854,0.213048,0.218145,0.211009,0.026504,0.022426


In [29]:
censusdata.printtable(censusdata.censustable('acs5', 2015, 'B25040'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B25040_001E  | B25040.  House Heating Fuel    | Total:                                                   | int  
B25040_002E  | B25040.  House Heating Fuel    | Utility gas                                              | int  
B25040_003E  | B25040.  House Heating Fuel    | Bottled, tank, or LP gas                                 | int  
B25040_004E  | B25040.  House Heating Fuel    | Electricity                                              | int  
B25040_005E  | B25040.  House Heating Fuel    | Fuel oil, kerosene, etc.                                 | int  
B25040_006E  | B25040.  House Heating Fuel    | Coal or coke                                             | int  
B25040_007E  | B25040.  House Heating Fuel    | Wood                                         

In [30]:

primary_heating_fuel_df_names = {}
for i in range(len(ca_county_lst)):
    df_name = 'primary_heating_fuel_df_county_' + str(ca_county_lst[i])
    
    df_data = censusdata.download('acs5', 2015,
                             censusdata.censusgeo([('state', '06'), ('county', ca_county_lst[i]), ('block group', '*')]),
                             ['B25040_002E', 'B25040_003E', 'B25040_004E', 'B25040_005E', 'B25040_006E',
                              'B25040_007E', 'B25040_008E', 'B25040_009E', 'B25040_010E'])
    
    df_data = df_data.rename(columns={'B25040_002E':'Utility Gas', 'B25040_003E':'Bottled, tank, or LP gas', 
                                      'B25040_004E':'Electricity', 'B25040_005E':'Fuel oil, kerosene, etc.', 
                                      'B25040_006E':'Coal or coke', 'B25040_007E':'Wood', 'B25040_008E':'Solar Energy',
                                      'B25040_009E':'Other Fuel', 'B25040_010E':'No Fuel Used'})
    df_data = df_data.div(df_data.sum(axis=1), axis=0)
        
    for i in range(len(df_data.index.values)):
        df_data.index.values[i] = str(df_data.index.values[i])[:31]
    
    primary_heating_fuel_df_names[df_name] = df_data
primary_heating_fuel_df_names['primary_heating_fuel_df_county_001']


,Utility Gas,"Bottled, tank, or LP gas",Electricity,"Fuel oil, kerosene, etc.",Coal or coke,Wood,Solar Energy,Other Fuel,No Fuel Used
"Block Group 4, Census Tract 409",0.600000,0.000000,0.342308,0.0,0.0,0.000000,0.000000,0.000000,0.057692
"Block Group 1, Census Tract 409",0.500000,0.000000,0.482353,0.0,0.0,0.000000,0.000000,0.017647,0.000000
"Block Group 2, Census Tract 409",0.839590,0.034130,0.126280,0.0,0.0,0.000000,0.000000,0.000000,0.000000
"Block Group 3, Census Tract 409",0.726872,0.000000,0.251101,0.0,0.0,0.022026,0.000000,0.000000,0.000000
"Block Group 1, Census Tract 409",0.847059,0.000000,0.152941,0.0,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
"Block Group 2, Census Tract 450",0.771384,0.029549,0.181960,0.0,0.0,0.017107,0.000000,0.000000,0.000000
"Block Group 1, Census Tract 450",0.676324,0.014985,0.267732,0.0,0.0,0.000000,0.015984,0.000000,0.024975
"Block Group 2, Census Tract 450",0.799296,0.000000,0.200704,0.0,0.0,0.000000,0.000000,0.000000,0.000000
"Block Group 1, Census Tract 450",0.784913,0.000000,0.215087,0.0,0.0,0.000000,0.000000,0.000000,0.000000


In [31]:
censusdata.printtable(censusdata.censustable('acs5', 2015, 'B11016'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B11016_001E  | B11016.  HOUSEHOLD TYPE BY HOU | Total:                                                   | int  
B11016_002E  | B11016.  HOUSEHOLD TYPE BY HOU | Family households:                                       | int  
B11016_003E  | B11016.  HOUSEHOLD TYPE BY HOU | !! Family households: 2-person household                 | int  
B11016_004E  | B11016.  HOUSEHOLD TYPE BY HOU | !! Family households: 3-person household                 | int  
B11016_005E  | B11016.  HOUSEHOLD TYPE BY HOU | !! Family households: 4-person household                 | int  
B11016_006E  | B11016.  HOUSEHOLD TYPE BY HOU | !! Family households: 5-person household                 | int  
B11016_007E  | B11016.  HOUSEHOLD TYPE BY HOU | !! Family households: 6-person household     

In [32]:
num_residents_df_names = {}
for i in range(len(ca_county_lst)):
    df_name = 'num_residents_df_county_' + str(ca_county_lst[i])
    
    df_data = censusdata.download('acs5', 2015,
                                  censusdata.censusgeo([('state', '06'), ('county', ca_county_lst[i]), ('block group', '*')]),
                                  ['B11016_003E', 'B11016_004E', 'B11016_005E', 'B11016_006E', 'B11016_007E', 'B11016_008E',
                                   'B11016_011E', 'B11016_012E', 'B11016_013E', 'B11016_014E', 'B11016_015E', 'B11016_016E'])
    
    df_data = df_data.rename(columns={'B11016_003E':'Family households: 2-person household', 'B11016_004E':'Family households: 3-person household',
                                      'B11016_005E':'Family households: 4-person household', 'B11016_006E':'Family households: 5-person household',
                                      'B11016_007E':'Family households: 6-person household', 'B11016_008E':'Family households: 7-or-more-person household',
                                      'B11016_010E':'Nonfamily households: 1-person household','B11016_011E':'Nonfamily households: 2-person household', 
                                      'B11016_012E':'Nonfamily households: 3-person household','B11016_013E':'Nonfamily households: 4-person household', 
                                      'B11016_014E':'Nonfamily households: 5-person household','B11016_015E':'Nonfamily households: 6-person household', 
                                      'B11016_016E':'Nonfamily households: 7-person household'})
    df_data = df_data.div(df_data.sum(axis=1), axis=0)
        
    for i in range(len(df_data.index.values)):
        df_data.index.values[i] = str(df_data.index.values[i])[:31]
    
    num_residents_df_names[df_name] = df_data
num_residents_df_names['num_residents_df_county_001']


,Family households: 2-person household,Family households: 3-person household,Family households: 4-person household,Family households: 5-person household,Family households: 6-person household,Family households: 7-or-more-person household,Nonfamily households: 2-person household,Nonfamily households: 3-person household,Nonfamily households: 4-person household,Nonfamily households: 5-person household,Nonfamily households: 6-person household,Nonfamily households: 7-person household
"Block Group 4, Census Tract 409",0.228070,0.157895,0.152047,0.011696,0.356725,0.093567,0.000000,0.000000,0.000000,0.0,0.0,0.0
"Block Group 1, Census Tract 409",0.421941,0.050633,0.400844,0.050633,0.000000,0.000000,0.075949,0.000000,0.000000,0.0,0.0,0.0
"Block Group 2, Census Tract 409",0.411911,0.186104,0.282878,0.049628,0.000000,0.069479,0.000000,0.000000,0.000000,0.0,0.0,0.0
"Block Group 3, Census Tract 409",0.505814,0.098837,0.000000,0.302326,0.000000,0.000000,0.093023,0.000000,0.000000,0.0,0.0,0.0
"Block Group 1, Census Tract 409",0.448087,0.147541,0.131148,0.071038,0.076503,0.000000,0.125683,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
"Block Group 2, Census Tract 450",0.288660,0.273196,0.249141,0.130584,0.005155,0.000000,0.037801,0.015464,0.000000,0.0,0.0,0.0
"Block Group 1, Census Tract 450",0.127729,0.267467,0.386463,0.045852,0.061135,0.000000,0.094978,0.000000,0.016376,0.0,0.0,0.0
"Block Group 2, Census Tract 450",0.407448,0.280394,0.207010,0.064622,0.000000,0.000000,0.040526,0.000000,0.000000,0.0,0.0,0.0
"Block Group 1, Census Tract 450",0.363399,0.397386,0.129412,0.071895,0.000000,0.000000,0.028758,0.000000,0.009150,0.0,0.0,0.0


In [33]:
censusdata.printtable(censusdata.censustable('acs5', 2015, 'B25010'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B25010_001E  | B25010.  Average Household Siz | !! Average household size -- Total:                      | int  
B25010_002E  | B25010.  Average Household Siz | !! Average household size -- Owner occupied              | int  
B25010_003E  | B25010.  Average Household Siz | !! Average household size -- Renter occupied             | int  
-------------------------------------------------------------------------------------------------------------------


In [34]:
num_residents_2_df = censusdata.download('acs5', 2015,
                             censusdata.censusgeo([('state', '06'), ('county', '001'), ('block group', '*')]),
                             ['B25010_001E', 'B25010_002E', 'B25010_003E'])

num_residents_2_df = num_residents_2_df.rename(columns={'B25010_001E':'Average Household Size', 'B25010_002E':'Owner Occupied',
                                                        'B25010_003E':'Renter Occupied'})

    
for i in range(len(df_data.index.values)):
    df_data.index.values[i] = str(df_data.index.values[i])[:31]
    
num_residents_2_df.head()

,Average Household Size,Owner Occupied,Renter Occupied
"Block Group 4, Census Tract 4097, Alameda County, California: Summary level: 150, state:06> county:001> tract:409700> block group:4",3.53,4.15,3.06
"Block Group 1, Census Tract 4098, Alameda County, California: Summary level: 150, state:06> county:001> tract:409800> block group:1",2.49,2.28,3.00
"Block Group 2, Census Tract 4098, Alameda County, California: Summary level: 150, state:06> county:001> tract:409800> block group:2",2.80,3.24,2.58
"Block Group 3, Census Tract 4098, Alameda County, California: Summary level: 150, state:06> county:001> tract:409800> block group:3",2.61,2.87,1.75
"Block Group 1, Census Tract 4099, Alameda County, California: Summary level: 150, state:06> county:001> tract:409900> block group:1",2.56,2.52,3.05


In [35]:
censusdata.printtable(censusdata.censustable('acs5', 2015, 'B19001'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B19001_001E  | B19001. Household Income in th | Total:                                                   | int  
B19001_002E  | B19001. Household Income in th | Less than $10,000                                        | int  
B19001_003E  | B19001. Household Income in th | $10,000 to $14,999                                       | int  
B19001_004E  | B19001. Household Income in th | $15,000 to $19,999                                       | int  
B19001_005E  | B19001. Household Income in th | $20,000 to $24,999                                       | int  
B19001_006E  | B19001. Household Income in th | $25,000 to $29,999                                       | int  
B19001_007E  | B19001. Household Income in th | $30,000 to $34,999                           

In [36]:

household_income_past_year_df_names = {}
for i in range(len(ca_county_lst)):
    df_name = 'household_income_past_year_df_county_' + str(ca_county_lst[i])
    
    df_data = censusdata.download('acs5', 2015,
                                  censusdata.censusgeo([('state', '06'), ('county', ca_county_lst[i]), ('block group', '*')]),
                                  ['B19001_002E', 'B19001_003E', 'B19001_004E', 'B19001_005E', 'B19001_006E',
                                  'B19001_007E', 'B19001_008E', 'B19001_009E', 'B19001_010E','B19001_011E', 'B19001_012E',
                                  'B19001_013E', 'B19001_014E','B19001_015E', 'B19001_016E', 'B19001_017E'])
    
    df_data = df_data.rename(columns={'B19001_002E':'Less than \$10,000', 'B19001_003E':'\$10,000 to \$14,999', 
                                      'B19001_004E':'\$15,000 to \$19,999', 'B19001_005E':'\$20,000 to \$24,999', 
                                      'B19001_006E':'\$25,000 to \$29,999',
                                      'B19001_007E':'\$30,000 to \$34,999', 'B19001_008E':'\$35,000 to \$39,999', 
                                      'B19001_009E':'\$40,000 to \$44,999', 'B19001_010E':'\$45,000 to \$49,999',
                                      'B19001_011E':'\$50,000 to \$59,999', 'B19001_012E':'\$60,000 to \$74,999',
                                      'B19001_013E':'\$75,000 to \$99,999', 'B19001_014E':'\$100,000 to \$124,999',
                                      'B19001_015E':'\$125,000 to \$149,999', 'B19001_016E':'\$150,000 to \$199,999',
                                      'B19001_017E':'\$200,000 or more'})
    df_data = df_data.div(df_data.sum(axis=1), axis=0)
        
    for i in range(len(df_data.index.values)):
        df_data.index.values[i] = str(df_data.index.values[i])[:31]
    
    household_income_past_year_df_names[df_name] = df_data
household_income_past_year_df_names['household_income_past_year_df_county_001']


,"Less than \$10,000","\$10,000 to \$14,999","\$15,000 to \$19,999","\$20,000 to \$24,999","\$25,000 to \$29,999","\$30,000 to \$34,999","\$35,000 to \$39,999","\$40,000 to \$44,999","\$45,000 to \$49,999","\$50,000 to \$59,999","\$60,000 to \$74,999","\$75,000 to \$99,999","\$100,000 to \$124,999","\$125,000 to \$149,999","\$150,000 to \$199,999","\$200,000 or more"
"Block Group 4, Census Tract 409",0.088462,0.076923,0.026923,0.000000,0.000000,0.073077,0.065385,0.042308,0.092308,0.076923,0.150000,0.200000,0.000000,0.030769,0.000000,0.076923
"Block Group 1, Census Tract 409",0.061765,0.000000,0.023529,0.000000,0.000000,0.000000,0.020588,0.035294,0.000000,0.000000,0.158824,0.167647,0.394118,0.050000,0.061765,0.026471
"Block Group 2, Census Tract 409",0.093857,0.266212,0.030717,0.047782,0.023891,0.093857,0.018771,0.023891,0.035836,0.010239,0.095563,0.163823,0.040956,0.034130,0.000000,0.020478
"Block Group 3, Census Tract 409",0.035242,0.035242,0.000000,0.096916,0.118943,0.000000,0.000000,0.070485,0.000000,0.000000,0.083700,0.096916,0.242291,0.171806,0.000000,0.048458
"Block Group 1, Census Tract 409",0.000000,0.000000,0.047059,0.047059,0.019608,0.043137,0.000000,0.000000,0.078431,0.031373,0.129412,0.227451,0.113725,0.000000,0.031373,0.231373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Block Group 2, Census Tract 450",0.010886,0.000000,0.013997,0.000000,0.029549,0.013997,0.000000,0.017107,0.021773,0.052877,0.076205,0.101089,0.110420,0.101089,0.279938,0.171073
"Block Group 1, Census Tract 450",0.004995,0.008991,0.012987,0.000000,0.025974,0.000000,0.000000,0.000000,0.000000,0.006993,0.049950,0.124875,0.091908,0.100899,0.189810,0.382617
"Block Group 2, Census Tract 450",0.043134,0.014085,0.029049,0.050176,0.000000,0.066901,0.019366,0.022887,0.000000,0.029930,0.017606,0.050176,0.097711,0.066901,0.229754,0.262324
"Block Group 1, Census Tract 450",0.023445,0.046891,0.063201,0.012232,0.024465,0.008155,0.077472,0.018349,0.012232,0.092762,0.056065,0.112130,0.115189,0.093782,0.120285,0.123344


In [37]:
dict_lst = [units_in_structure_df_names, contract_rent_df_names, year_structure_built_df_names, num_bedrooms_df_names,
           num_rooms_df_names, primary_heating_fuel_df_names, num_residents_df_names, household_income_past_year_df_names]


In [46]:
for i in dict_lst:
    for key, value in i.items():
        print(key, i[key].shape[1])

units_in_structure_df_county_001 8
units_in_structure_df_county_003 8
units_in_structure_df_county_005 8
units_in_structure_df_county_007 8
units_in_structure_df_county_009 8
units_in_structure_df_county_011 8
units_in_structure_df_county_013 8
units_in_structure_df_county_015 8
units_in_structure_df_county_017 8
units_in_structure_df_county_019 8
units_in_structure_df_county_021 8
units_in_structure_df_county_023 8
units_in_structure_df_county_025 8
units_in_structure_df_county_027 8
units_in_structure_df_county_029 8
units_in_structure_df_county_031 8
units_in_structure_df_county_033 8
units_in_structure_df_county_035 8
units_in_structure_df_county_037 8
units_in_structure_df_county_039 8
units_in_structure_df_county_041 8
units_in_structure_df_county_043 8
units_in_structure_df_county_045 8
units_in_structure_df_county_047 8
units_in_structure_df_county_049 8
units_in_structure_df_county_051 8
units_in_structure_df_county_053 8
units_in_structure_df_county_055 8
units_in_structure_d

In [47]:
total_rows = [1047,2,28,195,30,20,637,20,125,589,23,108,96,16,456,81,48,25,6425,80,175,17,79,144,12,17,233,106,75,1823,213,
22,1030,912,40,1092,1795,581,395,163,463,313,1075,196,131,5,37,285,387,308,62,43,13,270,48,430]
print(sum(total_rows))

23041


In [48]:
total_cols = [8, 24, 10, 6, 9, 9, 12, 16]
print(sum(total_cols))

94


In [40]:
units_in_structure_df_names['units_in_structure_df_county_001']

,"1, detached","1, attached",2,3 or 4,5 to 9,10 to 19,20 to 49,50 or more
"Block Group 4, Census Tract 409",0.619048,0.000000,0.000000,0.258503,0.122449,0.000000,0.000000,0.000000
"Block Group 1, Census Tract 409",1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
"Block Group 2, Census Tract 409",0.479472,0.000000,0.016129,0.209677,0.108504,0.057185,0.085044,0.043988
"Block Group 3, Census Tract 409",1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
"Block Group 1, Census Tract 409",0.819608,0.074510,0.000000,0.105882,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
"Block Group 2, Census Tract 450",0.896552,0.061782,0.000000,0.000000,0.000000,0.000000,0.020115,0.021552
"Block Group 1, Census Tract 450",0.615010,0.122807,0.000000,0.063353,0.112086,0.025341,0.000000,0.061404
"Block Group 2, Census Tract 450",0.693662,0.125000,0.000000,0.004401,0.014965,0.022887,0.056338,0.082746
"Block Group 1, Census Tract 450",0.614907,0.230849,0.052795,0.015528,0.008282,0.000000,0.037267,0.040373


In [41]:
variable_dct = {}

variable_dct_names = []

for i in dict_lst:
    name = re.search(r"(\w+df)", str(i)).group(0)
    variable_dct_names.append(name)
    starter_df = pd.DataFrame()
    for key, value in i.items():
        #i[key].reset_index(inplace=True)
        starter_df = pd.concat([starter_df, i[key]], axis=0)
    variable_dct[name] = starter_df

        
#variable_dct["units_in_structure_df"].shape

In [42]:
variable_dct_names

['units_in_structure_df',
 'contract_rent_df',
 'year_structure_built_df',
 'num_bedrooms_df',
 'num_rooms_df',
 'primary_heating_fuel_df',
 'num_residents_df',
 'household_income_past_year_df']

In [50]:
final_df = pd.DataFrame()
for i in variable_dct_names:
    final_df = pd.concat([final_df, variable_dct[i]], axis=1)

    
print(final_df.shape)
print(sum(total_rows), sum(total_cols))

final_df.head()

(23041, 94)
23041 94


,"1, detached","1, attached",2,3 or 4,5 to 9,10 to 19,20 to 49,50 or more,With cash rent: Less than \$100,With cash rent: \$100 to \$149,...,"\$35,000 to \$39,999","\$40,000 to \$44,999","\$45,000 to \$49,999","\$50,000 to \$59,999","\$60,000 to \$74,999","\$75,000 to \$99,999","\$100,000 to \$124,999","\$125,000 to \$149,999","\$150,000 to \$199,999","\$200,000 or more"
"Block Group 4, Census Tract 409",0.619048,0.00000,0.000000,0.258503,0.122449,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.065385,0.042308,0.092308,0.076923,0.150000,0.200000,0.000000,0.030769,0.000000,0.076923
"Block Group 1, Census Tract 409",1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.020588,0.035294,0.000000,0.000000,0.158824,0.167647,0.394118,0.050000,0.061765,0.026471
"Block Group 2, Census Tract 409",0.479472,0.00000,0.016129,0.209677,0.108504,0.057185,0.085044,0.043988,0.032663,0.032663,...,0.018771,0.023891,0.035836,0.010239,0.095563,0.163823,0.040956,0.034130,0.000000,0.020478
"Block Group 3, Census Tract 409",1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.070485,0.000000,0.000000,0.083700,0.096916,0.242291,0.171806,0.000000,0.048458
"Block Group 1, Census Tract 409",0.819608,0.07451,0.000000,0.105882,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.078431,0.031373,0.129412,0.227451,0.113725,0.000000,0.031373,0.231373


In [44]:
gross_rent_df_names = {}
for i in range(len(ca_county_lst)):
    df_name = 'gross_rent_df_county_' + str(ca_county_lst[i])
    
    df_data = censusdata.download('acs5', 2015,
                             censusdata.censusgeo([('state', '06'), ('county', ca_county_lst[i]), ('block group', '*')]),
                             ['B25063_003E', 'B25063_004E', 'B25063_005E', 'B25063_006E',
                              'B25063_007E', 'B25063_008E','B25063_009E', 'B25063_010E','B25063_011E', 'B25063_012E',
                              'B25063_013E', 'B25063_014E','B25063_015E', 'B25063_016E','B25063_017E', 'B25063_018E',
                              'B25063_019E', 'B25063_020E','B25063_021E', 'B25063_022E','B25063_023E', 'B25063_024E', 
                              'B25063_025E', 'B25063_026E'])
    
    df_data = df_data.rename(columns={'B25063_003E':'With cash rent: Less than \$100', 'B25063_004E':'With cash rent: \$100 to \$149',
                               'B25063_005E':'With cash rent: \$150 to \$199', 'B25063_006E':'With cash rent: \$200 to \$249',
                              'B25063_007E':'With cash rent: \$250 to \$299', 'B25063_008E':'With cash rent: \$300 to \$349',
                              'B25063_009E':'With cash rent: \$350 to \$399', 'B25063_010E':'With cash rent: \$400 to \$449',
                               'B25063_011E':'With cash rent: \$450 to \$499', 'B25063_012E':'With cash rent: \$500 to \$549',
                              'B25063_013E':'With cash rent: \$550 to \$599', 'B25063_014E':'With cash rent: \$600 to \$649',
                               'B25063_015E':'With cash rent: \$650 to \$699', 'B25063_016E':'With cash rent: \$700 to \$749',
                              'B25063_017E':'With cash rent: \$750 to \$799', 'B25063_018E':'With cash rent: \$800 to \$899',
                              'B25063_019E':'With cash rent: \$900 to \$999', 'B25063_020E':'With cash rent: \$1,000 to \$1,249',
                              'B25063_021E':'With cash rent: \$1,250 to \$1,499', 'B25063_022E':'With cash rent: \$1,500 to \$1,999 ',
                               'B25063_023E':'With cash rent: \$2,000 to \$2,499', 'B25063_024E':'With cash rent: \$2,500 to \$2,999', 
                              'B25063_025E':'With cash rent: \$3,000 to \$3,499', 'B25063_026E':'With cash rent: \$3,500 or more'})
    df_data = df_data.div(df_data.sum(axis=1), axis=0)
        
    for i in range(len(df_data.index.values)):
        df_data.index.values[i] = str(df_data.index.values[i])[:31]
    
    gross_rent_df_names[df_name] = df_data
gross_rent_df_names['gross_rent_df_county_001']

,With cash rent: Less than \$100,With cash rent: \$100 to \$149,With cash rent: \$150 to \$199,With cash rent: \$200 to \$249,With cash rent: \$250 to \$299,With cash rent: \$300 to \$349,With cash rent: \$350 to \$399,With cash rent: \$400 to \$449,With cash rent: \$450 to \$499,With cash rent: \$500 to \$549,...,With cash rent: \$750 to \$799,With cash rent: \$800 to \$899,With cash rent: \$900 to \$999,"With cash rent: \$1,000 to \$1,249","With cash rent: \$1,250 to \$1,499","With cash rent: \$1,500 to \$1,999","With cash rent: \$2,000 to \$2,499","With cash rent: \$2,500 to \$2,999","With cash rent: \$3,000 to \$3,499","With cash rent: \$3,500 or more"
"Block Group 4, Census Tract 409",0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.047297,0.000000,0.000000,...,0.000000,0.047297,0.000000,0.067568,0.445946,0.324324,0.000000,0.000000,0.000000,0.000000
"Block Group 1, Census Tract 409",0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.572917,0.041667,0.000000,0.385417,0.000000,0.000000
"Block Group 2, Census Tract 409",0.0,0.0,0.032663,0.052764,0.08794,0.0,0.0,0.000000,0.022613,0.027638,...,0.062814,0.241206,0.057789,0.193467,0.000000,0.050251,0.143216,0.000000,0.000000,0.000000
"Block Group 3, Census Tract 409",0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.200000,0.000000,...,0.000000,0.000000,0.000000,0.200000,0.000000,0.600000,0.000000,0.000000,0.000000,0.000000
"Block Group 1, Census Tract 409",0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Block Group 2, Census Tract 450",0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.229508,0.377049,0.000000,0.114754,0.131148,0.147541,0.000000
"Block Group 1, Census Tract 450",0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.038806,0.000000,0.026866,0.077612,0.376119,0.208955,0.077612,0.149254,0.000000
"Block Group 2, Census Tract 450",0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,...,0.045845,0.214900,0.103152,0.025788,0.045845,0.303725,0.017192,0.123209,0.000000,0.040115
"Block Group 1, Census Tract 450",0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.033654,0.403846,0.043269,0.024038,0.110577,0.235577,0.000000,0.149038,0.000000


In [45]:
contract_rent_df_names['contract_rent_df_county_001']

,With cash rent: Less than \$100,With cash rent: \$100 to \$149,With cash rent: \$150 to \$199,With cash rent: \$200 to \$249,With cash rent: \$250 to \$299,With cash rent: \$300 to \$349,With cash rent: \$350 to \$399,With cash rent: \$400 to \$449,With cash rent: \$450 to \$499,With cash rent: \$500 to \$549,...,With cash rent: \$750 to \$799,With cash rent: \$800 to \$899,With cash rent: \$900 to \$999,"With cash rent: \$1,000 to \$1,249","With cash rent: \$1,250 to \$1,499","With cash rent: \$1,500 to \$1,999","With cash rent: \$2,000 to \$2,499","With cash rent: \$2,500 to \$2,999","With cash rent: \$3,000 to \$3,499","With cash rent: \$3,500 or more"
"Block Group 4, Census Tract 409",0.000000,0.000000,0.0,0.067568,0.000000,0.000000,0.0,0.047297,0.000000,0.000000,...,0.000000,0.000000,0.020270,0.385135,0.256757,0.175676,0.000000,0.000000,0.000000,0.000000
"Block Group 1, Census Tract 409",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.614583,0.000000,0.000000,0.093750,0.291667,0.000000,0.000000
"Block Group 2, Census Tract 409",0.032663,0.032663,0.0,0.075377,0.032663,0.022613,0.0,0.000000,0.027638,0.090452,...,0.115578,0.133166,0.020101,0.173367,0.000000,0.165829,0.027638,0.000000,0.000000,0.000000
"Block Group 3, Census Tract 409",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.200000,0.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
"Block Group 1, Census Tract 409",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Block Group 2, Census Tract 450",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.606557,0.000000,0.000000,0.114754,0.278689,0.000000,0.000000
"Block Group 1, Census Tract 450",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.038806,0.000000,0.000000,0.026866,0.104478,0.349254,0.208955,0.152239,0.074627,0.000000
"Block Group 2, Census Tract 450",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.318052,0.000000,0.071633,0.103152,0.200573,0.140401,0.000000,0.000000,0.040115
"Block Group 1, Census Tract 450",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.033654,0.403846,0.067308,0.000000,0.206731,0.139423,0.110577,0.038462,0.000000
